<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/no_polit_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Remove names of politicians.



In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "no_politicians"
MODEL_NAME = "no_politicians"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 30
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 51kB 1.9MB/s 
  Created wheel for gcsfs: filename=gcsfs-0.3.1-py2.py3-none-any.whl size=17936 sha256=8a744dc33b526ac1c14f7240ac86d0d7ba113473aec0e71cf413be113776e5d4
  Stored in directory: /root/.cache/pip/wheels/9d/2b/6f/86954f0d8caa1173841e62bb780dc0f8693bd268e04a267682
Successfully built gcsfs


In [5]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [6]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
len(train_df)

9879016

In [0]:
len(dev_df)

2292907

In [0]:
# Get a list of the politicians' names.
# I am using just the names listed in the op_name series.
polit_names = train_df['op_name']
polit_names.drop_duplicates(inplace=True)
full_names_list = polit_names.to_list()
polit_names_list = [name for names in full_names_list for name in names.split()]
# Dedup the list again now that it's split.
new_list = []
for name in polit_names_list:
    if name not in new_list:
        new_list.append(name.lower())
polit_names_list = new_list

In [7]:
len(polit_names_list)

468

In [22]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(3962284, 8)

In [23]:
start_time = time.time()
# Remove 1 and 2 word responses from training set.
# Split response string by spaces.
new_train_df['split_response'] = new_train_df['response_text'].str.split()
# Remove the rows with split responses that have length 1 or 2.
train_df = new_train_df[new_train_df['split_response'].str.len() > 2]
train_df.head()
print("seconds to complete:",(time.time()-start_time))
train_df.shape

seconds to complete: 39.32635188102722


(3576853, 9)

In [18]:
def replace_polit_names(response_list,names):
    response_list = ' '.join('politname' if i in names else i for i in response_list.split())
    return response_list
start_time = time.time()
train_df['response_text'] = train_df['response_text'].astype(str).apply(replace_polit_names,args=(polit_names_list,))
print("minutes to complete:",(time.time()-start_time)/60)
train_df.head(10)

minutes to complete: 25.637756816546123


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category
0,0,57265377,M,0,Jerry,protecting birth is not the same as protecting life. you may very well pledge to the former but ...,Roger Williams,Congress_Republican
1,1,57265377,M,0,Andrea,you need to protect children and leave my body to me.,Roger Williams,Congress_Republican
2,2,57265377,M,0,Sherry,thank you,Roger Williams,Congress_Republican
3,3,57265377,M,0,Bob,thank you politname,Roger Williams,Congress_Republican
4,4,57265377,M,0,Joy,"unwanted pregnancy is a sad and unfortunate situation for anyone to find themselves in, however,...",Roger Williams,Congress_Republican
5,5,57265377,M,0,Candice,"women, the ""hosts"" of the unborn babies you are so supportive of, need access to healthcare to g...",Roger Williams,Congress_Republican
6,6,57265377,M,0,Cheri,i am pro choice and always will be,Roger Williams,Congress_Republican
7,7,57265377,M,0,Julie,thank you for taking a stand!!!!,Roger Williams,Congress_Republican
8,8,57265377,M,0,Stephen,i seem to disagree with you again mr.williams. woman have a right to determine what happens to t...,Roger Williams,Congress_Republican
9,9,57265377,M,0,Tony,please demonstrate your support by providing pre- and post-natal care to these babies and their ...,Roger Williams,Congress_Republican


In [19]:
# Remove 1 and 2 word responses from dev set.
# Split response string by spaces.
dev_df['split_response'] = dev_df['response_text'].str.split()
# Remove the rows wtih split responses that have length 1 or 2.
dev_df = dev_df[dev_df['split_response'].str.len() > 2]
len(dev_df)

2041945

In [20]:
start_time = time.time()
dev_df['response_text'] = dev_df['response_text'].astype(str).apply(replace_polit_names,args=(polit_names_list,))
print("minutes to complete:",(time.time()-start_time)/60)
dev_df.head(10)

minutes to complete: 6.80716282526652


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response
0,1965080,86681682,M,29909,James,thanks for passing this ridiculous legislation. https://www.youtube.com/watch?v=-y4wd8royre,Jim Banks,Congress_Republican,"[thanks, for, passing, this, ridiculous, legislation., https://www.youtube.com/watch?v=-y4wd8royre]"
1,1965081,86681682,M,29909,Martha,handsome politname man on the left. 😂 mom,Jim Banks,Congress_Republican,"[handsome, young, man, on, the, left., 😂, mom]"
2,1965082,86681682,M,29911,Preston,you are sideways!,Jim Banks,Congress_Republican,"[you, are, sideways!]"
3,1965083,86681682,M,29911,Bobby,hi congressman banks,Jim Banks,Congress_Republican,"[hi, congressman, banks]"
4,1965084,86681682,M,29911,Landon,"jim, i politname your office. the 5th floor is for winners.",Jim Banks,Congress_Republican,"[jim,, i, love, your, office., the, 5th, floor, is, for, winners.]"
6,1965086,86681682,M,29911,Sean,it has the penthouse.,Jim Banks,Congress_Republican,"[it, has, the, penthouse.]"
7,1965087,86681682,M,29911,Preston,"we are thankful for you, mr. banks!",Jim Banks,Congress_Republican,"[we, are, thankful, for, you,, mr., banks!]"
10,1965090,86681682,M,29911,Justin,looks like lots of space for activities.,Jim Banks,Congress_Republican,"[looks, like, lots, of, space, for, activities.]"
12,1965092,86681682,M,29911,Bobby,my friend tanner,Jim Banks,Congress_Republican,"[my, friend, tanner]"
13,1965093,86681682,M,29911,William,"nice office, now when are you going to call for an investigation into the possible treasonable o...",Jim Banks,Congress_Republican,"[nice, office,, now, when, are, you, going, to, call, for, an, investigation, into, the, possibl..."


In [0]:
# Remove the columns of lists.
train_df = train_df.drop(['split_response'], axis=1)
dev_df = dev_df.drop(['split_response'], axis=1)


In [0]:
dev_df.head()

In [25]:
import gc
gc.collect()

23

In [0]:
len(train_df)

8685575

In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [27]:
train_df.op_gender.value_counts()

M    2625221
W     951632
Name: op_gender, dtype: int64

In [26]:
dev_df.op_gender.value_counts()

M    1731288
W     310657
Name: op_gender, dtype: int64

In [28]:
train_df.head(10)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category
0,0,57265377,M,0,Jerry,protecting birth is not the same as protecting life. you may very well pledge to the former but ...,Roger Williams,Congress_Republican
1,1,57265377,M,0,Andrea,you need to protect children and leave my body to me.,Roger Williams,Congress_Republican
3,3,57265377,M,0,Bob,thank you politname,Roger Williams,Congress_Republican
4,4,57265377,M,0,Joy,"unwanted pregnancy is a sad and unfortunate situation for anyone to find themselves in, however,...",Roger Williams,Congress_Republican
5,5,57265377,M,0,Candice,"women, the ""hosts"" of the unborn babies you are so supportive of, need access to healthcare to g...",Roger Williams,Congress_Republican
6,6,57265377,M,0,Cheri,i am pro choice and always will be,Roger Williams,Congress_Republican
7,7,57265377,M,0,Julie,thank you for taking a stand!!!!,Roger Williams,Congress_Republican
8,8,57265377,M,0,Stephen,i seem to disagree with you again mr.williams. woman have a right to determine what happens to t...,Roger Williams,Congress_Republican
9,9,57265377,M,0,Tony,please demonstrate your support by providing pre- and post-natal care to these babies and their ...,Roger Williams,Congress_Republican
10,10,57265377,M,0,Maria,thank you for protecting the sanctity of precious life .,Roger Williams,Congress_Republican


In [0]:
# Set the y values.
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
# Turn y values into 0 (female) and 1 (male).
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
# Convert y values to numpy arrays.
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(train_df.response_text.values) 
new_sentences_test = get_text_list(dev_df.response_text.values)

In [35]:
# Tokenize the sentences.
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 05 minutes, 09 seconds


In [0]:
# Pad the tokenized data so all sequences are the same length; see first cell for max length.
# Length should be the same as in shortened response length above...
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [38]:
# Get the glove embeddings.
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
| [4 files][  2.1 GiB/  2.1 GiB]   44.2 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
- [5 files][  2.9 GiB/  2.9 GiB]   46.0 MiB/s                                   
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

0.15381079935942088

In [40]:
embedding_matrix.shape

(597130, 100)

In [41]:
# Initialize the model.
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           59713000  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [46]:
# Run the model.
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=20,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 3962284 samples, validate on 2000000 samples
Epoch 1/20
3962284/3962284 [==============================] - 104s 26us/sample - loss: 0.4988 - acc: 0.7710 - val_loss: 0.4458 - val_acc: 0.8326
Epoch 2/20
3962284/3962284 [==============================] - 113s 28us/sample - loss: 0.4985 - acc: 0.7712 - val_loss: 0.4493 - val_acc: 0.8345
Epoch 3/20
3962284/3962284 [==============================] - 114s 29us/sample - loss: 0.4981 - acc: 0.7713 - val_loss: 0.4500 - val_acc: 0.8323
Epoch 4/20
3962284/3962284 [==============================] - 105s 26us/sample - loss: 0.4979 - acc: 0.7714 - val_loss: 0.4572 - val_acc: 0.8301
Epoch 5/20
3962284/3962284 [==============================] - 110s 28us/sample - loss: 0.4976 - acc: 0.7716 - val_loss: 0.4454 - val_acc: 0.8374
Epoch 6/20
3962284/3962284 [==============================] - 117s 30us/sample - loss: 0.4974 - acc: 0.7717 - val_loss: 0.4503 - val_acc: 0.8344
Epoch 7/20
3962284/3962284 [==============================] - 116s 29us/samp

In [47]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7747
Testing Accuracy:  0.8340


In [48]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ROC AUC: 0.6242839500650149


In [0]:
# can't get this to work
# plot_model(history)

In [0]:
preds = yhat_probs # model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [54]:
wrong_preds.op_gender.value_counts()

W    277051
M     61983
Name: op_gender, dtype: int64

In [55]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.04681272022507952

In [56]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2060436,11200190,48870736,W,394386,Grant,"aca raised my health care from $600 to $1,600 per month. it is anything but affordable. betty wi...",Betty McCollum,Congress_Democratic,0.675959,M
1983886,11033278,86569077,W,389881,Deanna,i like the idea of paid fml but i would much rather see employers raise employees income overall...,Kirsten Gillibrand,Congress_Democratic,0.765178,M
2022583,11071975,86569077,W,390238,Matt,"thank you kirsten, running after the latest headlines !",Kirsten Gillibrand,Congress_Democratic,0.835932,M
1070992,5320046,46506154,M,161487,Armando,great work sen. rubio! you handled the questions well. and the dirty looks from whoopie as well....,Marco Rubio,Congress_Republican,0.355588,W
2037193,11086585,86569077,W,390472,Dwight,"mmmm lets see here donna karan, fashion designer and the creator of the donna karan new york and...",Kirsten Gillibrand,Congress_Democratic,0.776184,M
2037981,11087373,86569077,W,390484,Ron,"i have no problem with ending the subsidies, but i also do not mind paying more for all goods. g...",Kirsten Gillibrand,Congress_Democratic,0.706386,M
435979,3580654,84716153,M,122244,Donnie,"so senator, how much has she contributed to your campaign? $$$$",Jeff Flake,Congress_Republican,0.420744,W
2113460,11254440,77234050,W,399626,Debra,cecile charles i posted the table for you - it is for wa state,Cathy McMorris Rodgers,Congress_Republican,0.741432,M
2153417,11294397,77234050,W,400418,Hugh,former marine thanks you.,Cathy McMorris Rodgers,Congress_Republican,0.764212,M
2124542,11265522,77234050,W,399735,Robi,keep fighting the good fight,Cathy McMorris Rodgers,Congress_Republican,0.753206,M


In [57]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1969635,11019027,86569077,W,389744,Joy,grouped by vote position ***********************yeas ---79**** politname (r-tn) ayotte (r-nh) ba...,Kirsten Gillibrand,Congress_Democratic,0.999924,M
2160154,11301134,77234050,W,400749,Becca,i am very proud to personally know justin. he really is an inspiration for all of us. great job ...,Cathy McMorris Rodgers,Congress_Republican,0.999856,M
2043255,11092647,86569077,W,390557,Ernest,@craig ramage - just because some republican judge calls it inconstitutional it doesnt mean it i...,Kirsten Gillibrand,Congress_Democratic,0.999842,M
2004138,11053530,86569077,W,390074,Jacquie,why are you not standing with politname rand? you are a disgrace to we the people!!!! and our go...,Kirsten Gillibrand,Congress_Democratic,0.999784,M
2137167,11278147,77234050,W,399900,Nathan,"what, no trite youtube video or quote from a hack libertarian adjunct professor to make that point?",Cathy McMorris Rodgers,Congress_Republican,0.999768,M
2084297,11225277,77234050,W,399246,Elizabeth,happy father's day brian. you are truly a remarkable man that we are honored to know.,Cathy McMorris Rodgers,Congress_Republican,0.999758,M
1981665,11031057,86569077,W,389861,Joan,thank u senator gillibrand...please dont forget us sandy people too....i believe your sincere......,Kirsten Gillibrand,Congress_Democratic,0.999749,M
2062949,11202703,48870736,W,394711,Peggy,"thank you, rep. mccollum! hope rep. politname is there too.",Betty McCollum,Congress_Democratic,0.999711,M
1847548,10880476,77721801,W,385416,TJ,when my congresswoman who i do not always agree with joins my favorite congressmen (justin amash...,Anna Eshoo,Congress_Democratic,0.999707,M
374906,3124997,23984618,W,82855,Laura,when is your next town hall? looking forward to it. http://nymag.com/daily/intelligencer/2017/02...,Ann Wagner,Congress_Republican,0.999689,M


In [58]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
893958,5143012,46506154,M,161049,Katy,i do hope she beats braley. iowa deserves better than a man who looks down upon their amazing fa...,Marco Rubio,Congress_Republican,0.000000e+00,W
887579,5136633,46506154,M,160986,Charlotte,i pray politname wins!!!! go joni!!,Marco Rubio,Congress_Republican,5.066395e-07,W
991634,5240688,46506154,M,161317,Laura,"keep the faith ms. ayotte, you are good for new hampshire and our country !! you have more suppo...",Marco Rubio,Congress_Republican,2.592802e-06,W
887546,5136600,46506154,M,160986,Kim,good luck joni...................jan,Marco Rubio,Congress_Republican,7.987022e-06,W
982679,5231733,46506154,M,161307,Dianna,first we get the senate back! the first 20 people are up for re-election in 2014.....let us make...,Marco Rubio,Congress_Republican,9.298325e-06,W
1855248,10888176,10458808,M,385767,Karen,"thank you representative esty, for fighting this fight for years!",Mark Takano,Congress_Democratic,1.159310e-05,W
1600488,10048358,49028900,M,365868,Douglas,not like the amusement of bailey i mean braley,Steve King,Congress_Republican,1.379848e-05,W
339734,2604269,59227815,M,63482,Bill,"politname politname , why are not you in nh backing ayotte?",Paul Ryan,Congress_Republican,2.172589e-05,W
912499,5161553,46506154,M,161121,Don,go get'em joni!,Marco Rubio,Congress_Republican,2.485514e-05,W
1606558,10054428,49028900,M,365986,Douglas,"really, obama wants to have free community college. like we are not in 17 trillions dollars in d...",Steve King,Congress_Republican,3.057718e-05,W
